In [13]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import keras
from keras import backend as K
from keras.layers import Input, Embedding, LSTM, Dense, Conv2D
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Concatenate
from keras.layers import Conv3D, MaxPooling3D, Conv2D, MaxPooling2D
from keras.losses import categorical_crossentropy
from keras.layers.advanced_activations import PReLU, LeakyReLU
from keras.optimizers import Adadelta, Nadam, Adam
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
from tensorflow.python.client import device_lib
import utils
import imp
imp.reload(utils)

IMAGE_HEIGHT, IMAGE_WIDTH = 120, 120
AGE_CLASSES = 100
MAX_SURVIVAL = 2000

In [14]:
df = pd.read_csv("data/survival_data.csv")
df.head()
X = np.load('data/tumors_small_nz.npy')
X = X.transpose((0, 2, 1, 3, 4))

df['TSize'] = X[:, :3].sum(axis=(1, 2, 3, 4))
df['TSize1'] = X[:, 0:1].sum(axis=(1, 2, 3, 4))
df['TSize2'] = X[:, 1:2].sum(axis=(1, 2, 3, 4))
df['TSize3'] = X[:, 2:3].sum(axis=(1, 2, 3, 4))
df['BSize'] = X[:, 3:].sum(axis=(1, 2, 3, 4))

PATIENTS = 163
TUMOR_CLASSES = 3
TUMOR_SLICES = 129
for tumor_id in range(TUMOR_CLASSES):
    tumor_sums = X[:, tumor_id:tumor_id+1].sum(axis=(3, 4))
    for patient_id in range(PATIENTS):
        for slice_id in range(TUMOR_SLICES):
            df.loc[patient_id, "Tumor{}Slice{}Sum".format(tumor_id+1, slice_id)] = tumor_sums[patient_id, 0, slice_id]

"""Drop columns just containing 0"""
df = df.loc[:, (df != 0).any(axis=0)]
"Done"

'Done'

In [15]:
if "Brats17ID" in df.columns:
    _ = df.pop("Brats17ID")

Y_diff = df["Survival"].max() - df["Survival"].min()
Y_mean = df["Survival"].mean()
    
df = (df - df.mean()) / (df.max() - df.min())

if "Survival" in df.columns:
    Y = df.pop("Survival")

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=10, shuffle=True, random_state=42)

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest
from scipy.stats import randint as sp_randint
from sklearn.svm import SVR
import math

def backscale_Y(Y, Y_diff, Y_mean):
    return Y * Y_diff + Y_mean

In [ ]:
pca = PCA(n_components=50)
svm = SVR(kernel="sigmoid")

pipeline = Pipeline([("features", pca), ("svm", svm)])

param_dist = dict(features__n_components=list(range(40, 55, 1)),
                  svm__kernel=['sigmoid'],
                  svm__C=map(lambda x : x/float(100), list(range(80, 100, 5))))

grid_search = GridSearchCV(pipeline, param_grid=param_dist, verbose=0, n_jobs=-1, cv=10)
grid_search.fit(X_train, y_train)
print(grid_search.best_estimator_)

In [ ]:
#clf = Pipeline(steps=[('features', FeatureUnion(n_jobs=1,
#       transformer_list=[('pca', PCA(copy=True, iterated_power='auto', n_components=69, random_state=None,
#  svd_solver='auto', tol=0.0, whiten=False))],
#       transformer_weights=None)), ('svm', SVR(C=0.800735491526, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
#  gamma='auto', kernel='sigmoid', max_iter=-1, shrinking=False, tol=0.001,
#  verbose=False))])
#clf = Pipeline(steps=[('svm', SVR(C=0.800735491526, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
#  gamma='auto', kernel='sigmoid', max_iter=-1, shrinking=False, tol=0.001,
#  verbose=False))])

clf = grid_search.best_estimator_

scores = []

for train_index, test_index in kf.split(df):
    X_train, X_test = df.iloc[train_index, :], df.iloc[test_index, :]
    y_train, y_test = Y[train_index], Y[test_index]
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    backscaled_y_pred = backscale_Y(y_pred, Y_diff, Y_mean)
    backscaled_y_test = backscale_Y(y_test, Y_diff, Y_mean)
    score = math.floor(sum((backscaled_y_test-backscaled_y_pred)**2)/len(y_test))
    scores.append(score)
print sum(scores)/len(scores)